In [56]:
commonURL = ""
videos = commonURL + "Dynamic Tests/Originals"
saveInDir = commonURL + "Dynamic Tests/Results"

img_width, img_height = 96, 54
objetiveSize = (img_width, img_height)

In [57]:
mapVideo=[]
numTotalFrames = 4282
for i in range(0, numTotalFrames):
    if(i<381):
        mapVideo.append(0)
    else: 
        if(i<660):
            mapVideo.append(-1)
        else: 
            if(i<1300):
                mapVideo.append(1)
            else: 
                if(i<2900):
                    mapVideo.append(3)
                else: 
                    if(i<3963):
                        mapVideo.append(4)
                    else:
                        mapVideo.append(2)

In [58]:
from keras.models import load_model
model = load_model(commonURL + 'neuralNetworkSmall.h5')

In [59]:
def lookForBest(prediction):
    best = 0
    categories = 6
    for i,probability in enumerate(prediction[0]):
        if probability > prediction[0][best]:
            best = i
    return best

def predictImg(img,realLabel,model):
    prediction = model.predict(img)
    return lookForBest(prediction)

def translateNode(node):
    if node == 0:
        return "Hall - Resident Rooms"
    if node == 1:
        return "Building - Dinning room"
    if node == 2:
        return "Building - Rooms"
    if node == 3:
        return "Parking"
    if node == 4:
        return "Building - Gym"
    if node == 5:
        return "Gym interior"
    return "Unknown"

import time
def printAndRemove(msg):
    print(msg, end="\r")
    time.sleep(0)

In [62]:
import glob
import cv2
import os,shutil
import re
from keras.preprocessing import image as preprocessing
from PIL import Image, ImageFont, ImageDraw, ImageOps
import numpy as np


import matplotlib.pyplot as plt

for video in glob.glob(videos + "/*.mp4"):
    videoName = video.split('\\')[1]
    
    cap = cv2.VideoCapture(video)
    
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    try:
        os.stat('.temp')
    except:
        os.mkdir('.temp')
    totalTrues = 0
    frameNumber = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            printAndRemove("Working on frame nº" + str(frameNumber))
            
            # Preprocessing of the image for the Neural Network
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img).convert('RGB')
            img = img.rotate(180)
            img.thumbnail(objetiveSize, Image.ANTIALIAS)
            img = np.rot90(img)
            img = Image.fromarray(img)
            img = np.expand_dims(img, axis=0)
            
            labelPredicted = translateNode(lookForBest(model.predict(img)))
            

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = Image.fromarray(frame).convert('RGB')
            frame = frame.rotate(180)
            frameDraw = ImageDraw.Draw(frame)
            frameDraw.text( (20, 20), labelPredicted,  font = ImageFont.truetype("arial.ttf", 70), fill=(255,255,255))
            frame = np.array(frame)
            
            cv2.imwrite(".temp/frame" + str(frameNumber) + ".jpg", cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            
            # Accuracy
            if(translateNode(mapVideo[frameNumber]) == labelPredicted or mapVideo[frameNumber] == -1):
                totalTrues = totalTrues + 1
            frameNumber = frameNumber + 1
        else:
            break
            
    video = cv2.VideoWriter(videoName, cv2.VideoWriter_fourcc(*'DIVX'), 30.0, (frameWidth, frameHeight))

    def recompile(name):
        r= re.compile("(\d+).*").split(name)
        return [int(y) if y.isdigit() else y for y in r]
    
    filenames = glob.glob(".temp/*.jpg")
    filenames.sort(key=recompile)
    
    for image in filenames:
        video.write(cv2.imread(image))
    
    cap.release()
    cv2.destroyAllWindows()
    video.release()
    #shutil.rmtree('.temp/')
    print("")
    print("Video Ready")
    print("True Positives" + str(totalTrues))
    print("Number Frames" + str(frameNumber))
    print("Accuracy" + str(totalTrues/frameNumber))
"""
LEER_FOTOGRAMAS_VIDEO=True
video_name = './test/RecorridoFinal_etiquetado.mp4' #nombre y path del fichero DE SALIDA FINAL

# la ruta del video ORIGINAL. Por ejemplo es una carpeta llamada test
cam = cv2.VideoCapture("test/RecorridoFinal.mp4")

frameWidth = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))

try:
    os.stat('.temp')
except:
    os.mkdir('.temp')

currentframe = 0
ret=True
while (ret and LEER_FOTOGRAMAS_VIDEO):
    # obtener fotograma y condicion de parada
    ret, frame = cam.read()
    print "preparando frame" + str(currentframe+1)
    ########################################################################################
    #
    #
    # emplear aqui el metodo programado en en modulo correspondiente:
    #
    # etiqueta = knn.vecinoMasProximo(frame)
    #
    #
    #
    ########################################################################################
    cv2.putText(img=frame, text='Nodo Desconocido', org=(int(10), int(frameHeight-10)),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=2, color=(255, 255, 255))
    # final_frames.append(frame)
    cv2.imwrite(".temp/frame%d.jpg" %currentframe, frame)
    currentframe += 1

# creamos otro video con las imagenes etiquetadas
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'DIVX'), 30.0, (frameWidth, frameHeight))
print 'generando video'


def recompile(name):
    '''esta funcion es auxiliar para la ordenación de los nombres de los frames'''
    r= re.compile("(\d+).*").split(name)
    return [int(y) if y.isdigit() else y for y in r]


#cargar los frames temporalmente almacenados
filenames = glob.glob(".temp/*.jpg")

#reordenar segun el numero identificativo
filenames.sort(key=recompile)

#crear video frame a frame
[video.write(cv2.imread(image)) for image in filenames]
print 'Listo!'

# liberamos recursos
cv2.destroyAllWindows()
video.release()
cam.release()
shutil.rmtree('.temp/')
"""

Working on frame nº4281
Video Ready
True Positives1479
Number Frames4282
Accuracy0.3453993460999533


'\nLEER_FOTOGRAMAS_VIDEO=True\nvideo_name = \'./test/RecorridoFinal_etiquetado.mp4\' #nombre y path del fichero DE SALIDA FINAL\n\n# la ruta del video ORIGINAL. Por ejemplo es una carpeta llamada test\ncam = cv2.VideoCapture("test/RecorridoFinal.mp4")\n\nframeWidth = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))\nframeHeight = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))\n\ntry:\n    os.stat(\'.temp\')\nexcept:\n    os.mkdir(\'.temp\')\n\ncurrentframe = 0\nret=True\nwhile (ret and LEER_FOTOGRAMAS_VIDEO):\n    # obtener fotograma y condicion de parada\n    ret, frame = cam.read()\n    print "preparando frame" + str(currentframe+1)\n    ########################################################################################\n    #\n    #\n    # emplear aqui el metodo programado en en modulo correspondiente:\n    #\n    # etiqueta = knn.vecinoMasProximo(frame)\n    #\n    #\n    #\n    ########################################################################################\n    cv2.putText(img=frame,

In [68]:
import glob
import cv2
import os,shutil
import re
from keras.preprocessing import image as preprocessing
from PIL import Image, ImageFont, ImageDraw, ImageOps
import numpy as np


import matplotlib.pyplot as plt

for video in glob.glob(videos + "/*.mp4"):
    videoName = video.split('\\')[1]
    
    cap = cv2.VideoCapture(video)
    
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    totalTrues = 0
    frameNumber = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            print("Working on frame nº" + str(frameNumber))
            
            # Preprocessing of the image for the Neural Network
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img).convert('RGB')
            img = img.rotate(180)
            img.thumbnail(objetiveSize, Image.ANTIALIAS)
            img = np.rot90(img)
            img = Image.fromarray(img)
            img = np.expand_dims(img, axis=0)
            
            labelPredicted = translateNode(lookForBest(model.predict(img)))

            # Accuracy
            if(translateNode(mapVideo[frameNumber]) == labelPredicted or mapVideo[frameNumber] == -1):
                totalTrues = totalTrues + 1
            frameNumber = frameNumber + 1
            print("True Positives: " + str(totalTrues))
            print("Number Frames: " + str(frameNumber))
            print("Accuracy: " + str(totalTrues/frameNumber))
        else:
            break
            
    cap.release()
    cv2.destroyAllWindows()

Working on frame nº0
True Positives: 0
Number Frames: 1
Accuracy: 0.0
Working on frame nº1
True Positives: 0
Number Frames: 2
Accuracy: 0.0
Working on frame nº2
True Positives: 0
Number Frames: 3
Accuracy: 0.0
Working on frame nº3
True Positives: 0
Number Frames: 4
Accuracy: 0.0
Working on frame nº4
True Positives: 0
Number Frames: 5
Accuracy: 0.0
Working on frame nº5
True Positives: 0
Number Frames: 6
Accuracy: 0.0
Working on frame nº6
True Positives: 0
Number Frames: 7
Accuracy: 0.0
Working on frame nº7
True Positives: 0
Number Frames: 8
Accuracy: 0.0
Working on frame nº8
True Positives: 0
Number Frames: 9
Accuracy: 0.0
Working on frame nº9
True Positives: 0
Number Frames: 10
Accuracy: 0.0
Working on frame nº10
True Positives: 0
Number Frames: 11
Accuracy: 0.0
Working on frame nº11
True Positives: 0
Number Frames: 12
Accuracy: 0.0
Working on frame nº12
True Positives: 1
Number Frames: 13
Accuracy: 0.07692307692307693
Working on frame nº13
True Positives: 2
Number Frames: 14
Accuracy:

True Positives: 74
Number Frames: 99
Accuracy: 0.7474747474747475
Working on frame nº99
True Positives: 75
Number Frames: 100
Accuracy: 0.75
Working on frame nº100
True Positives: 76
Number Frames: 101
Accuracy: 0.7524752475247525
Working on frame nº101
True Positives: 77
Number Frames: 102
Accuracy: 0.7549019607843137
Working on frame nº102
True Positives: 78
Number Frames: 103
Accuracy: 0.7572815533980582
Working on frame nº103
True Positives: 79
Number Frames: 104
Accuracy: 0.7596153846153846
Working on frame nº104
True Positives: 80
Number Frames: 105
Accuracy: 0.7619047619047619
Working on frame nº105
True Positives: 81
Number Frames: 106
Accuracy: 0.7641509433962265
Working on frame nº106
True Positives: 82
Number Frames: 107
Accuracy: 0.7663551401869159
Working on frame nº107
True Positives: 83
Number Frames: 108
Accuracy: 0.7685185185185185
Working on frame nº108
True Positives: 84
Number Frames: 109
Accuracy: 0.7706422018348624
Working on frame nº109
True Positives: 85
Number 

True Positives: 165
Number Frames: 190
Accuracy: 0.868421052631579
Working on frame nº190
True Positives: 166
Number Frames: 191
Accuracy: 0.8691099476439791
Working on frame nº191
True Positives: 167
Number Frames: 192
Accuracy: 0.8697916666666666
Working on frame nº192
True Positives: 168
Number Frames: 193
Accuracy: 0.8704663212435233
Working on frame nº193
True Positives: 169
Number Frames: 194
Accuracy: 0.8711340206185567
Working on frame nº194
True Positives: 170
Number Frames: 195
Accuracy: 0.8717948717948718
Working on frame nº195
True Positives: 171
Number Frames: 196
Accuracy: 0.8724489795918368
Working on frame nº196
True Positives: 172
Number Frames: 197
Accuracy: 0.8730964467005076
Working on frame nº197
True Positives: 173
Number Frames: 198
Accuracy: 0.8737373737373737
Working on frame nº198
True Positives: 174
Number Frames: 199
Accuracy: 0.8743718592964824
Working on frame nº199
True Positives: 175
Number Frames: 200
Accuracy: 0.875
Working on frame nº200
True Positive

True Positives: 256
Number Frames: 281
Accuracy: 0.9110320284697508
Working on frame nº281
True Positives: 257
Number Frames: 282
Accuracy: 0.9113475177304965
Working on frame nº282
True Positives: 258
Number Frames: 283
Accuracy: 0.911660777385159
Working on frame nº283
True Positives: 259
Number Frames: 284
Accuracy: 0.9119718309859155
Working on frame nº284
True Positives: 260
Number Frames: 285
Accuracy: 0.9122807017543859
Working on frame nº285
True Positives: 261
Number Frames: 286
Accuracy: 0.9125874125874126
Working on frame nº286
True Positives: 262
Number Frames: 287
Accuracy: 0.9128919860627178
Working on frame nº287
True Positives: 263
Number Frames: 288
Accuracy: 0.9131944444444444
Working on frame nº288
True Positives: 264
Number Frames: 289
Accuracy: 0.9134948096885813
Working on frame nº289
True Positives: 265
Number Frames: 290
Accuracy: 0.9137931034482759
Working on frame nº290
True Positives: 266
Number Frames: 291
Accuracy: 0.9140893470790378
Working on frame nº291


True Positives: 308
Number Frames: 372
Accuracy: 0.8279569892473119
Working on frame nº372
True Positives: 309
Number Frames: 373
Accuracy: 0.8284182305630027
Working on frame nº373
True Positives: 310
Number Frames: 374
Accuracy: 0.8288770053475936
Working on frame nº374
True Positives: 310
Number Frames: 375
Accuracy: 0.8266666666666667
Working on frame nº375
True Positives: 310
Number Frames: 376
Accuracy: 0.824468085106383
Working on frame nº376
True Positives: 310
Number Frames: 377
Accuracy: 0.8222811671087533
Working on frame nº377
True Positives: 311
Number Frames: 378
Accuracy: 0.8227513227513228
Working on frame nº378
True Positives: 311
Number Frames: 379
Accuracy: 0.820580474934037
Working on frame nº379
True Positives: 311
Number Frames: 380
Accuracy: 0.8184210526315789
Working on frame nº380
True Positives: 311
Number Frames: 381
Accuracy: 0.8162729658792651
Working on frame nº381
True Positives: 312
Number Frames: 382
Accuracy: 0.8167539267015707
Working on frame nº382
T

...

True Positives: 1563
Number Frames: 3984
Accuracy: 0.39231927710843373
Working on frame nº3984
True Positives: 1563
Number Frames: 3985
Accuracy: 0.3922208281053952
Working on frame nº3985
True Positives: 1563
Number Frames: 3986
Accuracy: 0.3921224284997491
Working on frame nº3986
True Positives: 1563
Number Frames: 3987
Accuracy: 0.39202407825432656
Working on frame nº3987
True Positives: 1563
Number Frames: 3988
Accuracy: 0.39192577733199596
Working on frame nº3988
True Positives: 1563
Number Frames: 3989
Accuracy: 0.3918275256956631
Working on frame nº3989
True Positives: 1563
Number Frames: 3990
Accuracy: 0.39172932330827065
Working on frame nº3990
True Positives: 1563
Number Frames: 3991
Accuracy: 0.3916311701327988
Working on frame nº3991
True Positives: 1563
Number Frames: 3992
Accuracy: 0.3915330661322645
Working on frame nº3992
True Positives: 1563
Number Frames: 3993
Accuracy: 0.391435011269722
Working on frame nº3993
True Positives: 1563
Number Frames: 3994
Accuracy: 0.3913

True Positives: 1596
Number Frames: 4071
Accuracy: 0.3920412675018423
Working on frame nº4071
True Positives: 1597
Number Frames: 4072
Accuracy: 0.39219056974459726
Working on frame nº4072
True Positives: 1598
Number Frames: 4073
Accuracy: 0.39233979867419594
Working on frame nº4073
True Positives: 1599
Number Frames: 4074
Accuracy: 0.39248895434462444
Working on frame nº4074
True Positives: 1600
Number Frames: 4075
Accuracy: 0.39263803680981596
Working on frame nº4075
True Positives: 1601
Number Frames: 4076
Accuracy: 0.3927870461236506
Working on frame nº4076
True Positives: 1602
Number Frames: 4077
Accuracy: 0.39293598233995586
Working on frame nº4077
True Positives: 1603
Number Frames: 4078
Accuracy: 0.3930848455125061
Working on frame nº4078
True Positives: 1604
Number Frames: 4079
Accuracy: 0.3932336356950233
Working on frame nº4079
True Positives: 1605
Number Frames: 4080
Accuracy: 0.39338235294117646
Working on frame nº4080
True Positives: 1606
Number Frames: 4081
Accuracy: 0.3

True Positives: 1680
Number Frames: 4159
Accuracy: 0.4039432555902861
Working on frame nº4159
True Positives: 1681
Number Frames: 4160
Accuracy: 0.40408653846153847
Working on frame nº4160
True Positives: 1682
Number Frames: 4161
Accuracy: 0.40422975246335013
Working on frame nº4161
True Positives: 1683
Number Frames: 4162
Accuracy: 0.4043728976453628
Working on frame nº4162
True Positives: 1684
Number Frames: 4163
Accuracy: 0.4045159740571703
Working on frame nº4163
True Positives: 1685
Number Frames: 4164
Accuracy: 0.40465898174831894
Working on frame nº4164
True Positives: 1686
Number Frames: 4165
Accuracy: 0.40480192076830734
Working on frame nº4165
True Positives: 1687
Number Frames: 4166
Accuracy: 0.40494479116658666
Working on frame nº4166
True Positives: 1688
Number Frames: 4167
Accuracy: 0.4050875929925606
Working on frame nº4167
True Positives: 1689
Number Frames: 4168
Accuracy: 0.4052303262955854
Working on frame nº4168
True Positives: 1690
Number Frames: 4169
Accuracy: 0.40

True Positives: 1753
Number Frames: 4246
Accuracy: 0.4128591615638248
Working on frame nº4246
True Positives: 1753
Number Frames: 4247
Accuracy: 0.4127619496114905
Working on frame nº4247
True Positives: 1753
Number Frames: 4248
Accuracy: 0.4126647834274953
Working on frame nº4248
True Positives: 1753
Number Frames: 4249
Accuracy: 0.4125676629795246
Working on frame nº4249
True Positives: 1753
Number Frames: 4250
Accuracy: 0.41247058823529414
Working on frame nº4250
True Positives: 1753
Number Frames: 4251
Accuracy: 0.41237355916255
Working on frame nº4251
True Positives: 1753
Number Frames: 4252
Accuracy: 0.4122765757290687
Working on frame nº4252
True Positives: 1753
Number Frames: 4253
Accuracy: 0.41217963790265694
Working on frame nº4253
True Positives: 1753
Number Frames: 4254
Accuracy: 0.4120827456511519
Working on frame nº4254
True Positives: 1753
Number Frames: 4255
Accuracy: 0.41198589894242066
Working on frame nº4255
True Positives: 1753
Number Frames: 4256
Accuracy: 0.411889

AttributeError: 'str' object has no attribute 'release'